In [20]:
cd /content/drive/MyDrive/VIP_lab/Cancer-Net-PCa/Cancer-Net-PCa

/content/drive/MyDrive/VIP_lab/Cancer-Net-PCa/Cancer-Net-PCa


In [22]:
!pip install monai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.7 MB/s eta 0:00:00
